# Arbres rouge-noir

## Définitions

In [1]:
type 'a rb_tree = 
      E (* Vide *)
    | R of 'a * 'a rb_tree * 'a rb_tree (* noeud rouge *)
    | B of 'a * 'a rb_tree * 'a rb_tree (* noeud noir *)

type 'a rb_tree =
    E
  | R of 'a * 'a rb_tree * 'a rb_tree
  | B of 'a * 'a rb_tree * 'a rb_tree


In [2]:
let red = function (* la racine est-elle rouge ? *)
    | E | B(_, _, _) -> false
    | R(_, _, _) -> true

val red : 'a rb_tree -> bool = <fun>


In [3]:
let b = function (* rend la racine noire *)
    | R(r, g, d) -> B(r, g, d)
    | a -> a

val b : 'a rb_tree -> 'a rb_tree = <fun>


## Appartenance

In [4]:
let rec has e = function
    | E -> false
    | R(r, g, d) | B(r, g, d) -> 
        e = r || has e (if e < r then g else d);;

val has : 'a -> 'a rb_tree -> bool = <fun>


In [5]:
let a = B(1, R(0, E, E), R(3, E, E)) in
has 0 a && not (has 5 a) (* test *)

- : bool = true


## Rotations

In [6]:
let rotd = function
    | B(r, R(gr, gg, gd), d) -> R(gr, b gg, B(r, gd, d))
    | R(r, R(gr, gg, gd), d) -> R(gr, gg, R(r, gd, d))
    | _ -> failwith "error rotd";;

let rotg = function 
    | B(r, g, R(dr, dg, dd)) -> R(dr, B(r, g, dg), b dd)
    | R(r, g, R(dr, dg, dd)) -> R(dr, R(r, g, dg), dd)
    | _ -> failwith "error rotg";;

val rotd : 'a rb_tree -> 'a rb_tree = <fun>


val rotg : 'a rb_tree -> 'a rb_tree = <fun>


In [7]:
let rec balance t = let B(r, g, d) = t in
    match g, d with
  | R(gr, gg, gd), d when red gg -> rotd t
  | R(gr, gg, gd), d when red gd -> balance (B(r, rotg g, d)) (* rotation droite-gauche *)
  | g, R(dr, dg, dd) when red dd -> rotg t
  | g, R(dr, dg, dd) when red dg -> balance (B(r, g, rotd d)) (* rotation gauche-droite *)
  | _, _ -> t;;

File "[7]", line 1, characters 20-343:
1 | ....................let B(r, g, d) = t in
2 |     match g, d with
3 |   | R(gr, gg, gd), d when red gg -> rotd t
4 |   | R(gr, gg, gd), d when red gd -> balance (B(r, rotg g, d)) (* rotation droite-gauche *)
5 |   | g, R(dr, dg, dd) when red dd -> rotg t
6 |   | g, R(dr, dg, dd) when red dg -> balance (B(r, g, rotd d)) (* rotation gauche-droite *)
7 |   | _, _ -> t..
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
(E|R (_, _, _))


val balance : 'a rb_tree -> 'a rb_tree = <fun>


In [8]:
let add t e =
    let rec aux = function
      | E -> R(e, E, E)
      | B(r, g, d) when e < r -> balance (B(r, aux g, d))
      | R(r, g, d) when e < r -> R(r, aux g, d)
      | B(r, g, d) -> balance (B(r, g, aux d))
      | R(r, g, d) -> R(r, g, aux d) in
    aux t |> b (* on rend la racine noire *)

val add : 'a rb_tree -> 'a -> 'a rb_tree = <fun>


## Visualisation

In [9]:
let tree_to_tex ?(file_out="out") t =
    let open Printf in
    let file_out = sprintf "%s.tex" file_out in
    let f = open_out file_out in
    fprintf f "\\documentclass[convert={outfile=\\jobname.png}]{standalone}\n
\\usepackage{tikz}
\\usepackage{forest}\n
\\begin{document}\n
\\tikzset{every node/.style={draw, circle}}\n
\\begin{forest}\n";
    let rec dfs t = match t with
        | R(r, g, d) | B(r, g, d) ->
            let c = if red t then "red" else "black" in
            fprintf f "[ %i, %s " r c;
            dfs g; dfs d;
            fprintf f "]\n"
        | E -> () in
    dfs t;
    fprintf f ";\n\\end{forest}\n\\end{document}\n";
    close_out f;
    let _ = sprintf "lualatex %s 1> /dev/null; rm -f *.aux *.log *.texa" file_out
    |> Sys.command in ()

val tree_to_tex : ?file_out:string -> int rb_tree -> unit = <fun>


In [10]:
let swap t i j =
    let tmp = t.(i) in
    t.(i) <- t.(j);
    t.(j) <- tmp

let shuffle t =
    for i = 0 to Array.length t - 1 do
        swap t i (Random.int (i+1))
    done

let a = Array.init 14 (fun i -> i);;

Array.fold_left add E a 
|> tree_to_tex ~file_out:"example_sorted";;

shuffle a;;
Array.fold_left add E a 
|> tree_to_tex ~file_out:"example_random";;

val swap : 'a array -> int -> int -> unit = <fun>


val shuffle : 'a array -> unit = <fun>


val a : int array = [|0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13|]


- : unit = ()


- : unit = ()


- : unit = ()


In [11]:
let t = Array.fold_left add E a in
tree_to_tex ~file_out:"example_random" t;
tree_to_tex ~file_out:"example_random_del" (del 8 t);

error: compile_error